'''
To do:

load dicom data

convert to nifti

run totalsegmentator - pleural effusion

save segmentation

qualitative examination in slicer

get sørensen-dice score between GT and totalSeg 
'''

In [56]:

from preprocess_tools import *
import os
import glob
import SimpleITK as sitk
from tqdm import tqdm



dataset_path = '/Lungeholdet2024/PleuraEffusion_pilot/Dataset/manifest-1603198545583/NSCLC-Radiomics'

def convertDCM(DCM_file_name: str, output_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(DCM_file_name)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()

    # Added a call to PermuteAxes to change the axes of the data
    image = sitk.PermuteAxes(image, [2, 1, 0])

    sitk.WriteImage(image, f'{output_path}.nii.gz')


In [69]:
files = glob.glob(os.path.join(dataset_path, "*1-0[0-4][0-9]*", "**", "**", "[0-9]-[0-9][0-9]*.dcm"))

files = [f.replace('\\', '/') for f in files]
folder_set = set()
for folder in files:
    fold = folder.split('/1-')[0]
    folder_set.add(fold)
len(folder_set)

49

In [63]:
for folder in tqdm(folder_set):
    convertDCM(folder,output_path=f'/Lungeholdet2024/PleuraEffusion_pilot/Dataset/nifti_data/{folder[85:94]}')


100%|██████████| 49/49 [02:03<00:00,  2.53s/it]


In [70]:
for folder in tqdm(folder_set):
    get_segmentations(f'/Lungeholdet2024/PleuraEffusion_pilot/Dataset/nifti_data/{folder[85:94]}.nii.gz',
                      f'/Lungeholdet2024/PleuraEffusion_pilot/Dataset/total_pleural_segmentation/{folder[85:94]}.nii.gz',
                      task='pleural_pericard_effusion', fast=False)

  0%|          | 0/49 [00:00<?, ?it/s]c:\Users\PC\anaconda3\envs\Lungeholdet\lib\site-packages\nnunetv2\inference\predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experime

In [71]:
effusion_gt_path = '\Lungeholdet2024\PleuraEffusion_pilot\Effusions'
files = glob.glob(os.path.join(effusion_gt_path, "*1-0[0-4][0-9]*","*first_reviewer.*"))

files = [f.replace('\\', '/') for f in files]
gt_effusion_set = set()
for folder in files:
    fold = folder.split('/1-')[0]
    gt_effusion_set.add(fold)
gt_effusion_set

{'/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-001/LUNG1-001_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-002/LUNG1-002_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-005/LUNG1-005_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-008/LUNG1-008_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-013/LUNG1-013_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-016/LUNG1-016_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-018/LUNG1-018_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-024/LUNG1-024_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-026/LUNG1-026_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-028/LUNG1-028_eff

In [145]:
dice_coefficients = []
for lung_patient in tqdm(gt_effusion_set):
    totalseg = load_nifti_convert_to_numpy(f'/Lungeholdet2024/PleuraEffusion_pilot/Dataset/total_pleural_segmentation/{lung_patient[48:57]}'+'.nii.gz')
    totalseg = totalseg.transpose(2,1,0)
    gt_seg = load_nifti_convert_to_numpy(lung_patient)
    totalseg = np.where(totalseg == 2,1,0)
    im1 = np.asarray(totalseg).astype(np.bool_)
    im2 = np.asarray(gt_seg).astype(np.bool_)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    dice_coefficient = 2. * intersection.sum() / (im1.sum() + im2.sum())
    dice_coefficients.append([lung_patient[48:57],dice_coefficient])
    print(f'The dice coefficient of patient {lung_patient[55:57]} is {dice_coefficient}')

100%|██████████| 14/14 [04:17<00:00, 18.37s/it]


In [146]:
dice_coefficients

[['LUNG1-026', 0.0],
 ['LUNG1-005', 0.005750684632918495],
 ['LUNG1-038', 0.0],
 ['LUNG1-008', 0.00047337089803232843],
 ['LUNG1-001', 0.0],
 ['LUNG1-035', 0.0],
 ['LUNG1-042', 0.0],
 ['LUNG1-046', 0.0],
 ['LUNG1-013', 0.0],
 ['LUNG1-016', 0.0],
 ['LUNG1-028', 0.0],
 ['LUNG1-024', 0.016665542953368858],
 ['LUNG1-002', 0.0],
 ['LUNG1-018', 0.0]]

In [91]:
f'/Lungeholdet2024/PleuraEffusion_pilot/Dataset/total_pleural_segmentation/{list(gt_effusion_set)[0][48:57]}'+'.nii.gz'
list(gt_effusion_set)[0][48:57]


'LUNG1-026'